In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [3]:
import logging
import logging.config
import configparser
import os
import time
from pathlib import Path
import signal

import lmsquery
from waveshare_epd import epd5in83

In [4]:
import cfg
import epdlib

In [5]:
# this works best as a global variable
logConfig = Path(cfg.LOGCONFIG)
logging.config.fileConfig(logConfig.absolute())
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

logger = logging.getLogger(__name__)

In [6]:
def configuration(configFile=None):
    
    configDefaults = cfg.CONFIGDEFAULTS
    
    if not configFile:
         configFile = Path(cfg.CONFIGFILE)

    config = configparser.ConfigParser()
    logger.info(f'reading configuration: {configFile}')
    config.read(configFile)
    
    try:
        for section in configDefaults:
            if section not in config.sections():
                logger.debug(f'adding section: {section}')
                config.add_section(section)
            
            for option in configDefaults[section]:
                if not config.has_option(section, option):
                    logger.debug(f'missing option: {option}')
                    logger.debug(f'setting {option} to: {configDefaults[section][option]}')
                    config[section][option] = str(configDefaults[section][option])
            with open(configFile, 'w') as file:
                config.write(file)
    
    except Exception as e:
        logging.exception(f'exception on configuration file: {e}')
        raise
    
    logger.debug(f'config file contains sections: {config.sections()}')
    return config

In [7]:
def query(lms, last=0, delay=7):
    '''query the player only when a specified delay has passed
    Accepts:
        last: float - last time query was called

    Returns:
        tuple(last, lms.now_playing()
    '''
    if last==0:
        last = time.clock_gettime(time.CLOCK_MONOTONIC)-delay

    if time.clock_gettime(time.CLOCK_MONOTONIC) > last+delay:
        return time.clock_gettime(time.CLOCK_MONOTONIC), lms.now_playing()

    else:
        return last, None

In [8]:
class signalHandler(object):
    '''handle specific signals and allow graceful exiting while loop
    https://stackoverflow.com/a/31464349/5530152
    
    Signals Handled Gracefully:
        SIGINT
        SIGTERM
    Atributes:
        kill_now (bool) default: False
    '''
    kill_now = False
    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)
    
    def exit_gracefully(self, signum, frame):
        self.kill_now = True

In [14]:
def main():
    '''main entry point
    '''   
    logger.setLevel(logging.DEBUG)
    logger.info('Starting program')
    config = configuration()
    # scan for lms server and use the first one (this may be a terrible idea for other people)
#     lmsServer = lmsquery.scanLMS()[0]
    
    # define LMS query object
#     lms = lmsquery.LMSQuery(lmsServer['host'], lmsServer['port'], config.get('server', 'player'))
    lms = lmsquery.LMSQuery(player_id=config.get('server', 'player'))
    
    screen = epdlib.Screen()
    screen.epd = epd5in83.EPD()
    screen.initEPD()
    screen.clearEPD()
    layout = epdlib.Layout(layout=epdlib.layouts.threeRow)
    layout.update_contents(query(lms, 0)[1])
    screen.initEPD()
    screen.elements=layout.blocks.values()
    screen.concat()
    screen.writeEPD()    
    
    
    sigHandler = signalHandler()
    
    # last update
    updated = 0
    # id of currently playing track
    nowPlayingID = None
    # status of player 
    nowPlayingMode = None

    # now playing fields to display
    fields = ['title', 'album', 'artist', 'mode']
    
    
    print(f'pid: {os.getpid()}')
    try:
        while not sigHandler.kill_now:
            updated, value = query(lms, updated)
            response = value
            if response:
                if response['id'] != nowPlayingID or response['mode'] != nowPlayingMode:
                    
                    layout.update_contents(value)
                    screen.initEPD()
                    screen.elements=layout.blocks.values()
                    screen.concat()
                    screen.writeEPD()                      
                    
                    nowPlayingID = response['id']
                    nowPlayingMode = response['mode']
                    for field in fields:
                        print(f'{field}: {response[field]}')
                    print('\n')
            time.sleep(0.5)
    finally:
        print("cleaning up")
    

In [ ]:
if __name__ == '__main__':
    
    foo = main()

__main__         5:     INFO - Starting program
__main__         9:     INFO - reading configuration: servercfg.ini
__main__         30:     DEBUG - config file contains sections: ['server', 'screen']
root         69:     DEBUG - e-Paper busy
root         72:     DEBUG - e-Paper busy release
root         69:     DEBUG - e-Paper busy
root         72:     DEBUG - e-Paper busy release
root         92:     DEBUG - {'__name__': 'epdlib.Layout', '__doc__': None, '__package__': 'epdlib', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0xb2ab8ef0>, '__spec__': ModuleSpec(name='epdlib.Layout', loader=<_frozen_importlib_external.SourceFileLoader object at 0xb2ab8ef0>, origin='/home/pi/src/slimpi_epd/epdlib/Layout.py'), '__file__': '/home/pi/src/slimpi_epd/epdlib/Layout.py', '__cached__': '/home/pi/src/slimpi_epd/epdlib/__pycache__/Layout.cpython-37.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: Non

root         173:     DEBUG - calculating values from layout for resolution (600, 448)
root         203:     DEBUG - ***title***
root         109:     DEBUG - checking key/values
root         114:     DEBUG - missing key: dimensions; adding and setting to None
root         210:     DEBUG - dimensions: (600, 179)
root         235:     DEBUG - has explict position
root         237:     DEBUG - abs_coordinates: (0, 0)
root         135:     DEBUG - calculating font size
root         136:     DEBUG - using font at path: /home/pi/src/slimpi_epd/fonts/Open_Sans/OpenSans-Regular.ttf
root         146:     DEBUG - target Y fontsize: 62.65
root         159:     DEBUG - fontsize: 59
root         203:     DEBUG - ***coverart***
root         109:     DEBUG - checking key/values
root         114:     DEBUG - missing key: dimensions; adding and setting to None
root         210:     DEBUG - dimensions: (360, 269)
root         220:     DEBUG - has calculated position
root         237:     DEBUG - abs_co

root         348:     DEBUG - h-center line: How Long?
root         356:     DEBUG - h-center image coordinates
root         361:     DEBUG - v-center image coordinates
root         365:     DEBUG - image coordinates (150, 50)
root         295:     DEBUG - updating block: artist
root         307:     DEBUG - formatted list:
 ['Vampire', 'Weekend']
root         336:     DEBUG - text image dimensions: (131, 73)
root         361:     DEBUG - v-center image coordinates
root         365:     DEBUG - image coordinates (360, 187)
root         298:     DEBUG - ignoring block coverid
root         298:     DEBUG - ignoring block duration
root         298:     DEBUG - ignoring block album_id
root         298:     DEBUG - ignoring block genre
root         295:     DEBUG - updating block: album
root         307:     DEBUG - formatted list:
 ['Father of the', 'Bride']
root         336:     DEBUG - text image dimensions: (180, 66)
root         361:     DEBUG - v-center image coordinates
root         

In [11]:
l = epdlib.Layout(layout=epdlib.layouts.threeRow)

root         92:     DEBUG - {'__name__': 'epdlib.Layout', '__doc__': None, '__package__': 'epdlib', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0xb2ab8ef0>, '__spec__': ModuleSpec(name='epdlib.Layout', loader=<_frozen_importlib_external.SourceFileLoader object at 0xb2ab8ef0>, origin='/home/pi/src/slimpi_epd/epdlib/Layout.py'), '__file__': '/home/pi/src/slimpi_epd/epdlib/Layout.py', '__cached__': '/home/pi/src/slimpi_epd/epdlib/__pycache__/Layout.cpython-37.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in f

root         210:     DEBUG - dimensions: (600, 179)
root         235:     DEBUG - has explict position
root         237:     DEBUG - abs_coordinates: (0, 0)
root         135:     DEBUG - calculating font size
root         136:     DEBUG - using font at path: /home/pi/src/slimpi_epd/fonts/Open_Sans/OpenSans-Regular.ttf
root         146:     DEBUG - target Y fontsize: 62.65
root         159:     DEBUG - fontsize: 59
root         203:     DEBUG - ***coverart***
root         109:     DEBUG - checking key/values
root         114:     DEBUG - missing key: dimensions; adding and setting to None
root         210:     DEBUG - dimensions: (360, 269)
root         220:     DEBUG - has calculated position
root         237:     DEBUG - abs_coordinates: (0, 0)
root         203:     DEBUG - ***artist***
root         109:     DEBUG - checking key/values
root         114:     DEBUG - missing key: dimensions; adding and setting to None
root         210:     DEBUG - dimensions: (240, 90)
root         220

In [ ]:
globals()